# In Class Lab 1

### Due by midnight, thursday in your github repository 'Labs/Lab1' folder


In [1]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants

### Astropy Units:  https://docs.astropy.org/en/stable/units/index.html


## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [2]:
def VLSR(Ro, mu=6.379, vpec=12.24*u.km/u.s):
    '''
    Calculates the velocity at the local standard of rest.
    VLSR = 4.74*mu*Ro - vpec
    -----
    Parameters: 
        Ro: Solar radius in kpc (astropy units).
        mu: Proper motion of Sgr A* in mas/yr. 
            Default from Reid & Brunthaler 2004.
        vpec: peculiar motion of the sun in the v_sun direction in km/s (astropy units).
              Default from Schonrich 2010.
    -----
    Outputs: 
        V_LSR: The local standard of rest in km/s (astropy units)
    '''
    V_LSR = 4.74*mu*(Ro/u.kpc)*u.km/u.s - vpec
    return V_LSR

In [3]:
#Different values of distance from galctic center
Ro_reid = 8.34*u.kpc #Water maser (Reid 2014 ApJ 783)
Ro_abuter = 8.178*u.kpc #GRAVITY collaboration (Abuter+2019 A&A 625)
Ro_sparke = 7.9*u.kpc #Sparke & Gallagher textbook

In [4]:
#Calculate VLSR using Reid 2014
VLSR_reid = VLSR(Ro_reid)
print(VLSR_reid)

239.9320764 km / s


In [7]:
#Calculate VLSR using Abuter+2019
VLSR_abuter = VLSR(Ro_abuter)
print(np.round(VLSR_abuter))

235.0 km / s


In [6]:
#Calculate VLSR using Sparke & Gallagher
VLSR_sparke = VLSR(Ro_sparke) 
print(VLSR_sparke)

226.628034 km / s


### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [8]:
def TorbSun(Ro, vcirc):
    '''
    Computes the orbital period of the sun:
    T = 2 pi R / v

    Paramaters:
        Ro (astropy quantity): distance to the Galactic Center from
            the sun (kpc)
        vcirc (astropy quantity): velocity of the sun in the "v" 
            direction (km/s)
    
    Returns:
        T (astropy quantity): orbital period (Gyr)
    '''

    v_kpcGyr = vcirc.to(u.kpc/u.Gyr) #converting vcirc to kpc/Gyr
    T = 2*np.pi*Ro/v_kpcGyr #calculating orbital period

    return T


In [9]:
v_sunpec = 12.24*u.km/u.s #peculiar motion of sun

In [10]:
vsun = VLSR_abuter + v_sunpec #total motion of the sun in the "v" direction

In [11]:
#orbital period of the sun
T_abuter = TorbSun(Ro_abuter, vsun)
print(T_abuter)

0.20318680562272234 Gyr


### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [13]:
age_univ = 13.8*u.Gyr #age of the universe in Gyr
print(age_univ/T_abuter) #number of rotations about the GC

67.91779593023313


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  M$_\odot$) ? 

In [14]:
print(const.G)

  Name   = Gravitational constant
  Value  = 6.6743e-11
  Uncertainty  = 1.5e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2018


In [15]:
Grav = const.G.to(u.kpc**3/u.Gyr**2/u.Msun) #convert G to useful units
print(Grav)

4.498502151469554e-06 kpc3 / (solMass Gyr2)


In [16]:
#Density profile is rho = VLSR^2/(4pi*G*R^2)
#Mass (r) = integrate rho dV
#         = integrate rho 4pi*r^2*dr
#         = integrate rho VLSR^2/(4pi*G*R^2)  4pi*r^2*dr
#         = integrate VLSR^2/G dr (flat rotation curve for Isothermal Sphere)
#         = VLSR^2/G r

def massIso(r, VLSR):
    '''
    Computes the dark matter mass enclosed within a given
    distance r assuming an Isothermal Sphere model:
    M(r) = VLSR^2/G * r

    Parameters:
        r (astropy quantity): distance from the Galactic
            Center (kpc)
        VLSR (astropy quantity): velocity of the Local
            Standard of Rest (km/s)
    
    Returns:
        M (astropy quantity): mass enclosed within r 
            (Msun)
    '''

    VLSR_kpcGyr = VLSR.to(u.kpc/u.Gyr) #converting to kpc/Gyr
    M = (VLSR_kpcGyr**2/Grav)*r #Isothermal Sphere Mass Profile

    return M

In [20]:
#compute mass enclosed within Ro (GRAVITY collab)
M_IsoSolar = massIso(Ro_abuter, VLSR_abuter)
print(f'{M_IsoSolar:.2e}')

1.05e+11 solMass


In [22]:
#compute mass enclosed within 260 kpc
M_IsoSolar = massIso(260*u.kpc, VLSR_abuter)
print(f'{M_IsoSolar:.2e}')

3.34e+12 solMass


## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [33]:
#Potential for Hernquist Sphere
#   Phi = -GM/(r+a)
#Escape speed:
#   vesc^2 = 2GM/(r+a)
#rearrange for M:
#   M = vesc^2/(2G/(r+a))

def massHernVesc(vesc, r, a=30*u.kpc):
    '''
    Determines the total dark matter mass needed given an
    escape speed, assuming a Hernquist profile:
    M = vesc^2/(2G/(r+a))

    Parameters:
        vesc (astropy quantity): escape speed (or speed of 
            satellite) (km/s)
        r (astropy quantity): distance from Galactic Center
            (kpc)
        a (astropy quantity): Hernquist scale length (kpc)
            Default is 30 kpc
    
    Returns:
        M (astropy quantity): mass within r (Msun)
    '''

    vsec_kpcGyr = vesc.to(u.kpc/u.Gyr) #converting to kpc/Gyr
    M = vsec_kpcGyr**2/2/Grav*(r+a) # mass contained in r

    return M

In [34]:
v_leo = 196*u.km/u.s #speed if Leo I, Sohn et al.
r = 260*u.kpc #distance from GC

In [35]:
M_LeoI = massHernVesc(v_leo, r)
print(f'{M_LeoI:.2e}')

1.30e+12 solMass
